<a href="https://colab.research.google.com/github/RamsesMDLC/Smolagent_Project_1/blob/main/Smolagents_Project_1_YT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. LOADING LIBRARIES / MODULES / CLASSES**



In [1]:
#Installs the smolagents library along with extensions defined in the [toolkit] option.
!pip install smolagents[toolkit]

#Components from smolagents
  #CodeAgent: The "agent". It orchestrates reasoning and tool usage.
  #DuckDuckGoSearchTool: The "tool". It lets the agent fetch information from the web.
  #TransformersModel: It "allow us to get access to the model through Hugging Face". A wrapper for Hugging Face Transformer models.
from smolagents import CodeAgent, DuckDuckGoSearchTool, TransformersModel

#API key
  #Provides a secure way to access stored secrets (like API tokens) within Google Colab.
from google.colab import userdata
  #Allows programmatic login to Hugging Face Hub.
from huggingface_hub import login

#Tokenizer: class in the Hugging Face Transformers library to process text inputs ("prompts or text") and outputs ("answer") for the model.
  #This means AutoTokenizer forms the bridge:
    #Input text → tokens/tensors → Model
      #Splitting text into tokens (smaller pieces such as words or subwords).
      #Converting these tokens into numbers ("tensors"), called input IDs, which the model uses for computation.
      #Managing extra elements like special tokens (e.g., [CLS], [SEP], padding).
    #Model output tokens/tensors → decoded text
  #It automatically loads and configures the correct tokenizer for a specified model (i.e., there’s no need to know the model-specific tokenizer class).
from transformers import AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


#**2. GETTING THE TOKEN**

In [2]:
# Securely get Hugging Face token and login
hf_token = userdata.get('HF_TOKEN')
if hf_token:
    login(hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Token not found. Please add HF_TOKEN secret.")

Successfully logged in to Hugging Face!


#**3. DEFINING MODEL / TOKENIZER / PAD**


In [3]:
#Defining Model (from Hugging Face)
model_id = "Qwen/Qwen1.5-1.8B"

#Initialize tokenizer
  #Load a pretrained tokenizer for the given model identified by model_id (in this case "Qwen/Qwen1.5-1.8B")
    #The tokenizer includes vocabulary, tokenization rules, special tokens, and associated settings needed to convert raw text into token IDs.
tokenizer = AutoTokenizer.from_pretrained(model_id)

#Check whether the tokenizer has a designated padding token.
  #Padding tokens are used to make all input sequences the same length by adding special "pad" tokens to shorter sequences.
    #Real-world texts vary in length, so to batch-process multiple sequences efficiently, shorter sequences are padded with these special tokens until they match the longest sequence length in the batch.
    #Padding tokens carry no meaningful information and are meant only to fill space for model input consistency.
  #Padding tokens ensure stable input preprocessing and model compatibility during tokenization and generation
  #If the padding token is not set, the tokenizer or model might throw errors during inference or training.
  #By assigning the EOS (end-of-sentence) token as padding, the code ensures compatibility even when a dedicated pad token is not defined for the particular model.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the model
model = TransformersModel(model_id=model_id)

# Fix pad_token_id in model config if not set
if model.model.config.pad_token_id is None:
    model.model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

**Log output (Tokenizer Files Being Downloaded)**

Hugging Face automatically downloads the tokenizer artifacts from the repository Qwen/Qwen1.5-1.8B. These include:

* tokenizer_config.json (1.29kB): Contains metadata about the tokenizer, such as whether it lowercases text, what type of tokenizer is being used (e.g., BPE, SentencePiece), and any special tokens (CLS, SEP, etc.).

* vocab.json (2.78MB): Stores the vocabulary mapping for the tokenizer: a dictionary linking each token string to an ID (e.g., "elephant" -> 5031). This is used to convert text into token IDs.

* merges.txt (1.67MB): Defines the merge operations for a Byte Pair Encoding (BPE) tokenizer. It describes how characters and subwords are combined into larger tokens.

* tokenizer.json (7.03MB): A consolidated JSON file that includes vocabulary, merges, and tokenizer configuration in one place. This is often faster and more convenient to load.

**Log output (Model Files Being Downloaded)**

Hugging Face automatically downloads the model artifacts. These include:

* config.json (662B): Stores model architecture parameters (e.g., hidden size, number of layers, attention heads, max position embeddings).

* model.safetensors (3.67GB): The actual pretrained model weights in the safetensors format (preferred over .bin for safety and efficiency). This file makes up the majority of the size because it contains billions of parameters.

* generation_config.json (138B): Stores default parameters for text generation, like temperature, top-k, top-p, max length, and repetition penalty.

#**4. PROCESSING THE INPUT AND GENERATING AN OUTPUT**


In [4]:
#Initialize agent
  #CodeAgent: A higher-level wrapper that can run a language model while also using external tools.
  #tools=[DuckDuckGoSearchTool()]: The agent can access web search via DuckDuckGo, so it can pull in fresh information to supplement the model’s reasoning.
  #model=model: The agent is linked to a pretrained transformer-based language model (e.g., Hugging Face’s transformers).
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

#Prepare input text and tokenize with attention mask
  #input_text: The user’s query in plain text.
  #tokenizer: Converts text into tokens (numerical IDs) suitable for the model.
    #return_tensors="pt": Returns PyTorch tensors (instead of lists or arrays).
    #padding=True: Ensures uniform sequence lengths by filling shorter sequences with padding tokens.
    #truncation=True: Ensures long sequences are cut down to fit the model’s input size.

#IMPORTANT: The input is transformed from TEXT to NUMBERS
input_text = "How long would it take for an elephant to cross the United States from Florida to California?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

#The tokenization produces a dictionary inputs containing (these are extracted separately for clarity, and will be used in the generation call):
  #input_ids: Token IDs for the sequence.
  #attention_mask: A binary mask (1s for real tokens, 0s for padding) that tells the model which positions to attend to.
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

#IMPORTANT: The input (in NUMBERS format) is transformed into output (NUMBERS format)

#Since agent.run() might not allow passing attention_mask directly; therefore, call the model generation:
  #model.model: Accesses the underlying transformer model inside the agent wrapper.
  #generate: Method used for text generation (e.g., greedy search, beam search, sampling depending on model’s default). This produces generated_ids, which are the token IDs for the generated text.
    #input_ids, attention_mask: Ensure the model knows exactly which parts of the input are real text vs padding.
    #pad_token_id: In case the model needs to pad internally, this specifies the right padding token ID.
    #max_new_tokens=50: Limits how many new tokens the model can generate beyond the input.
generated_ids = model.model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    pad_token_id=model.model.config.pad_token_id,
    max_new_tokens=50
)

#IMPORTANT: The output (in NUMBERS format) is transformed into output (TEXT format)

# Decode generated tokens
  #tokenizer.decode: Converts the generated token IDs back into a readable string.
  #skip_special_tokens=True: Removes artifacts like <pad>, <eos> that the model may produce.
  #Prints the final model response.
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:")
print(generated_text)

Generated text:
How long would it take for an elephant to cross the United States from Florida to California? An elephant would need to travel approximately 2,000 miles to cross the United States from Florida to California. This would take approximately 100 days, assuming that the elephant is able to travel at a consistent speed of 30
